# Locations extraction

## Part 1: Regex

Links locations in standardized format directly to cities with linked lat/lng (using USCities dataset)

In [33]:
# import required packages and read in datasets

import numpy as np
import pandas as pd
import spacy
import re
import datetime
import dateutil.parser as parser

doc_catalog = pd.read_csv('document_catalog_20250815.csv')
uscities = pd.read_csv('uscities.csv')

In [34]:
doc_catalog.columns

Index(['ID', 'Box', 'title', 'Document Date', 'Document Length',
       'Manuscript Type', 'Collection', 'Source', 'Internal Notes',
       'Workflow Stage', 'Object Type', 'Author', 'Keywords', 'Head',
       'Document Top', 'Dateline', 'Salutation', 'Document Body', 'Signature',
       'Post Script', 'Document Extra', 'Marginalia', 'Enclosure', 'Footnote',
       'General', 'Image Identifier', 'Response'],
      dtype='object')

In [35]:
# ideal would be if Series was a part of the export

In [37]:
doc_catalog['Object Type'].unique()

array([nan, 'Letter', 'Pamphlet', 'Article', 'Speech', 'Essay',
       'Ephemera', 'Interview', 'Report', 'Legal'], dtype=object)

In [41]:
speeches = doc_catalog[(doc_catalog['Object Type'] == 'Speech') | (doc_catalog['Object Type'] == 'Ephemera')]

In [43]:
speeches['Workflow Stage'].value_counts()

Workflow Stage
Early Access    396
Proof read       60
Cataloged        13
Transcribed      13
Digitized         3
Tandem read       2
Name: count, dtype: int64

In [57]:
speeches[speeches['Workflow Stage'] == 'Transcribed']

,ID,Box,title,Document Date,Document Length,Manuscript Type,Collection,Source,Internal Notes,Workflow Stage,...,Document Body,Signature,Post Script,Document Extra,Marginalia,Enclosure,Footnote,General,Image Identifier,Response
1877,7,NaN,"Address at the ""New Directions in Civil Rights...","Mon, 05/04/1998 - 12:00",NaN,NaN,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NaN,Transcribed,...,This is the twentieth anniversary year of the ...,NaN,NaN,NaN,NaN,NaN,"The Sixties: Years of Hope—Days of Rage, by To...",NaN,NaN,NaN
2280,498,Box 10 Folder 5,Reference Materials for Speech concerning the ...,"Fri, 01/01/1999 - 12:00\n - Fri, 12/31/1999 - ...",260.0,Printed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two records associat...,Transcribed,...,[The following are reference materials for Jul...,NaN,NaN,NaN,NaN,NaN,* Member of the New York Bar. Former General C...,NaN,"3242561, 3242562, 3242563, 3242564, 3242565, 3...",NaN
2450,603,Box 2 Folder 22,Speech about Politics and the Ways Policies Ig...,"Thu, 01/01/1970 - 12:00\n - Thu, 12/31/1970 - ...",20.0,"Draft, Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Transcribed,...,"[1970]\nBrothers and sisters, African people, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2451,602,Box 2 Folder 23,Speech about Politics and the Ways Policies Ig...,"Thu, 01/01/1970 - 12:00\n - Thu, 12/31/1970 - ...",6.0,"Copy, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Transcribed,...,Julian Bond\nOne supposes that almost every Am...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2887,1117,Box 12 Folder 8,"Constituent List for Newsletter Distribution, ...","Wed, 06/28/1967 - 12:00",1.0,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: Please scan each page of this s...,Transcribed,...,"Mr. C. H. Aderhold 121 Chestnut Street, SW Atl...",NaN,NaN,NaN,NaN,NaN,NaN,From Julian Bond to 136th House District const...,NaN,NaN
3718,1921,Box 13 Folder 4,Memo from Julian Bond to call Mers. Gerado Eub...,"Mon, 09/09/1968 - 12:00",1.0,Handwritten,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: You may remove the staple and r...,Transcribed,...,Atlant\nCall Mrs Gerado Eubanks (ask Dr. H.M. ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"To Julian Bond from Katherine Camp, 9 Sept 196..."
3792,2004,Box 13 Folder 5,"Radnor Junior High School ""Go Series"" Business...","Mon, 01/01/1968 - 12:00\n - Tue, 12/31/1968 - ...",1.0,Printed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: It looks like you'll need to re...,Transcribed,...,John J. TramaCo-chairman(215) 692-2680\nDonald...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3855,2081,Box 13 Folder 6,From Julian Bond to Georgia Secretary of State...,"Tue, 10/08/1968 - 12:00",1.0,"Autograph, Copy, Handwritten, Printed, Signed,...",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",NaN,Transcribed,...,[Image]: inner circle: Three pillars supportin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3908,2117,Box 13 Folder 6,"Julian Bond's Tentative Schedule, 30 Oct 1968","Wed, 10/30/1968 - 12:00",1.0,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: There are two separate items in...,Transcribed,...,"JULIAN BOND - Tentative Schedule\nWednesday, O...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4089,2307,Box 13 Folder 7,Handwritten Note ca. Nov 1968,"Fri, 11/01/1968 - 12:00\n - Sat, 11/30/1968 - ...",1.0,Handwritten,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Date based on location at the end of box 13 fo...,Transcribed,...,Clark Call Center for Study 

In [56]:
# identified mistake I made in proofreading
# fixing it here so I don't have to export the doc catalog again
speeches.loc[2363, 'Workflow Stage'] = 'Proof read'

In [105]:
state_abbrev_to_name = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
    "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
    "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
    "WI": "Wisconsin", "WY": "Wyoming", "D.C.": "District of Columbia", "DC": "District of Columbia"
}

state_name_to_abbrev = {v: k for k, v in state_abbrev_to_name.items()}

uscities['state_abbrev'] = uscities['state_name'].map(state_name_to_abbrev)

# Step 3: Generate all combinations
location_variants = set()

for _, row in uscities.iterrows():
    city = row['city']
    state = row['state_name']
    abbrev = row['state_abbrev']
    
    # location_variants.add(city)
    location_variants.add(f"{city}, {state}")
    location_variants.add(f"{city}, {abbrev}")

# Step 4: Escape and build regex pattern
escaped_locations = [re.escape(loc) for loc in location_variants]
pattern = r'\b(?:' + '|'.join(escaped_locations) + r')\b'

# Step 5: Test text
title = "Top Tech Events in San Francisco, Austin, and Miami, FL"

# Find matches
matches = re.findall(pattern, title, flags=re.IGNORECASE)
print(matches)

['Miami, FL']


In [59]:
uscities.drop(['zips', 'id'], axis=1, inplace=True)

In [63]:
uscities['City'] = uscities['city'] + ', ' + uscities['state_name']

# Extract the city name using str.extract()
speeches['city'] = speeches['title'].str.extract(f'({pattern})', expand=False)

C:\Users\charl\AppData\Local\Temp\ipykernel_16836\1639426336.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speeches['city'] = speeches['title'].str.extract(f'({pattern})', expand=False)


In [64]:
sum(speeches.city.value_counts())

108

In [69]:
regex_cities = speeches[speeches.city.notna()]

def replace_state_abbreviations(text):
    # Compile a regex pattern that matches all two-letter state abbreviations as whole words
    pattern = r'\b(' + '|'.join(state_abbrev_to_name.keys()) + r')\b'
    return re.sub(pattern, lambda m: state_abbrev_to_name[m.group()], text)
    
regex_cities.city = regex_cities.city.apply(replace_state_abbreviations)

C:\Users\charl\AppData\Local\Temp\ipykernel_16836\71271229.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex_cities.city = regex_cities.city.apply(replace_state_abbreviations)


In [72]:
regex_cities.city.value_counts()

city
Atlanta, Georgia              34
New Orleans, Louisiana         6
Los Angeles, California        4
Jackson, Mississippi           4
Houston, Texas                 4
Birmingham, Alabama            3
Cleveland, Ohio                3
Charlotte, North Carolina      3
St. Louis, Missouri            2
Kansas City, Missouri          2
New York, New York             2
Sacramento, California         2
Chicago, Illinois              2
Minneapolis, Minnesota         2
Pittsburgh, Pennsylvania       2
Seattle, Washington            2
Philadelphia, Pennsylvania     2
Montgomery, Alabama            2
Baltimore, Maryland            2
Des Moines, Iowa               2
Sanford, Florida               2
Wichita, Kansas                1
San Juan, Puerto Rico          1
Fort Collins, Colorado         1
Greensboro, North Carolina     1
Topeka, Kansas                 1
Arlington, Texas               1
Macon, Georgia                 1
Honolulu, Hawaii               1
Gary, Indiana                  1
Cedar

In [73]:
speeches_no_regex = speeches[~speeches['ID'].isin(regex_cities['ID'])]

In [74]:
len(speeches)

487

In [77]:
len(speeches_no_regex), len(regex_cities)

(379, 108)

## Part 2: NLP

Uses NLP to attempt to target other relevant locations

In [79]:
nlp = spacy.load('en_core_web_sm')

possible_locations = []

for idx, row in speeches_no_regex.iterrows():
    text = row['title']
    doc = nlp(text)
    locations = [ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]]
    possible_locations.append(locations)

In [81]:
speeches_no_regex['possible_cities'] = possible_locations

C:\Users\charl\AppData\Local\Temp\ipykernel_16836\1952855365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  speeches_no_regex['possible_cities'] = possible_locations


In [87]:
speeches_possible = speeches_no_regex[speeches_no_regex.possible_cities.apply(len) > 0]
len(speeches_possible)

79

In [89]:
for idx, row in speeches_possible.iterrows():
    print(idx, row['title'], row['possible_cities'])

339 (Draft) Speech Introductory Remarks on Zimbabwe, 6 Dec 1973 ['Zimbabwe']
340 Enclosure: Speech Introductory Remarks on Zimbabwe, 6 Dec 1973 ['Zimbabwe']
1873 Speech on the health of the United States education system, 1973 February 2 ['the United States']
1875 DUPLICATE DO NOT PUBLISH: "Civil Rights and Wrongs" Planned Parenthood National Atlantic Region Spring Conference, Washington, D.C., May 20, 1989 ['Washington', 'D.C.']
1881 Ephemera/Research for Speech at Press Conference in Washington concerning the Vietnam War and the Bach Mai Hospital Emergency Fund, 1973 January 2 (2 of 2) ['Washington']
1882 Speech at Press Conference in Washington concerning the Vietnam War and the Bach Mai Hospital Emergency Fund, 1973 January 2 (1 of 2) ['Washington']
1898 Commencement Address at Federal City College, Washington, D.C., 1973 June 17 ['Washington', 'D.C.']
1915 Speech concerning South Africa [Probably 1978] ['South Africa']
1923 Duplicate of ID120: Speech concerning South Africa, 1973 

In [91]:
# export to excel and manually update - too many

speeches_possible.to_excel('speeches_possible.xlsx')

In [99]:
speeches_no_regex = pd.read_excel('speeches_possible.xlsx')

In [100]:
speeches_no_regex.drop(['possible_cities', 'Unnamed: 0'], axis = 1, inplace = True)

In [102]:
speeches_no_regex = speeches_no_regex[speeches_no_regex.city.notna()]
len(speeches_no_regex)

45

In [107]:
speeches_no_regex.city = speeches_no_regex.city.apply(replace_state_abbreviations)

In [108]:
speeches_no_regex.city.value_counts()

city
Washington, District of Columbia       8
Atlanta, Georgia                       6
Capahosic, Virginia                    4
Philadelphia, Pennsylvania             4
Grambling, Louisiana                   3
Villanova, Pennsylvania                2
Tuskegee, Alabama                      2
Great Barrington, Massachusetts        2
Grenada                                2
Prince Edward County, Virginia         2
Lincoln University, Pennsylvania       1
New York, New York                     1
Wilberforce, Ohio                      1
Newcastle-upon-Tyne, United Kingdom    1
Hampton, Virginia                      1
Charlottesville, Virginia              1
Mount Laurel, New Jersey               1
Berea, Kentucky                        1
Miami, Florida                         1
Chicago, Illinois                      1
Name: count, dtype: int64

## Part 3: Merge

Merge and export the two datasets. Will split by year and convert to mapped tile layer using ArcGIS Online Notebooks.

In [114]:
regex_cities = regex_cities.reset_index().drop('index', axis = 1)

In [115]:
all_located_speeches = pd.concat([regex_cities, speeches_no_regex])

In [139]:
city_data = uscities[['City', 'lat', 'lng']]
city_data.columns = ['city', 'lat', 'lng']
city_data.index = city_data.city
city_data = city_data.drop('city', axis = 1)

In [142]:
# we will clean up the meat of it later

all_located_speeches = all_located_speeches.join(city_data, how = 'left', on = 'city')

In [144]:
sum(all_located_speeches.lat.isna())

12

In [146]:
all_located_speeches[all_located_speeches.lat.isna()]

,ID,Box,title,Document Date,Document Length,Manuscript Type,Collection,Source,Internal Notes,Workflow Stage,...,Document Extra,Marginalia,Enclosure,Footnote,General,Image Identifier,Response,city,lat,lng
43,427,Box 9 Folder 24,Speech concerning the Civil Rights Movement an...,"Thu, 05/07/1998 - 12:00\n - Sun, 05/10/1998 - ...",23.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",This is the only document in the folder.\nImag...,Early Access,...,NaN,NaN,NaN,"1. Quoted in the New York Times, March 18, 199...",NaN,"3240466, 3240467, 3240468, 3240469, 3240470, 3...",NaN,"Newcastle-upon-Tyne, United Kingdom",NaN,NaN
50,458,Box 1 Folder 25,Address at the Dedication of W.E.B. Du Bois Me...,"Sat, 10/18/1969 - 12:00",6.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: There are several items in this...,Early Access,...,NaN,"[Handwritten marginalia on page 1, seemingly u...",NaN,NaN,NaN,"3201887, 3201888, 3201889, 3201890, 3201891, 3...",NaN,"Great Barrington, Massachusetts",NaN,NaN
51,502,Box 10 Folder 8,Printed Version of Commencement Address at St....,"Fri, 05/12/2000 - 12:00",12.0,Printed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two items in this fo...,Early Access,...,NaN,NaN,NaN,NaN,Commencement Address at St. George's Universit...,"3242462, 3242463, 3242464, 3242465, 3242466, 3...",NaN,Grenada,NaN,NaN
52,501,Box 10 Folder 8,Commencement Address at St. George's Universit...,"Fri, 05/12/2000 - 12:00",12.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two items in this fo...,Early Access,...,NaN,NaN,NaN,"1. ""Appeal to the United Nations Found Meeting...",NaN,"3242474, 3242475, 3242476, 3242477, 3242478, 3...",NaN,Grenada,NaN,NaN
53,498,Box 10 Folder 5,Reference Materials for Speech concerning the ...,"Fri, 01/01/1999 - 12:00\n - Fri, 12/31/1999 - ...",260.0,Printed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two records associat...,Transcribed,...,NaN,NaN,NaN,* Member of the New York Bar. Former General C...,NaN,"3242561, 3242562, 3242563, 3242564, 3242565, 3...",NaN,"Prince Edward County, Virginia",NaN,NaN
54,497,Box 10 Folder 5,Speech concerning the celebration of the anniv...,"Fri, 01/01/1999 - 12:00\n - Fri, 12/31/1999 - ...",14.0,"Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two records associat...,Early Access,...,NaN,NaN,NaN,"1. Green v. County School Board, 391 U.S. 430,...",Reference Materials for Speech concerning the ...,"3242821, 3242822, 3242823, 3242824, 3242825, 3...",NaN,"Prince Edward County, Virginia",NaN,NaN
57,514,Box 10 Folder 16,Speech concerning the National Housing Act of ...,"Sat, 09/15/2001 - 12:00",7.0,Typed,Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There is one item in this fold...,Early Access,...,NaN,NaN,NaN,"i See, e.g., Huntington Branch, NAACP v. Hunti...",NaN,"3242290, 3242291, 3242292, 3242293, 3242294, 3...",NaN,"Mount Laurel, New Jersey",NaN,NaN
68,589,Box 2 Folder 13,Speech concerning Black Voters and Officials d...,"Fri, 07/24/1970 - 12:00",6.0,"Handwritten, Typed, Version",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Note: There are three items in this f...,Proof read,...,NaN,NaN,NaN,NaN,Speech concerning Black Voters and Officials d...,"3202577, 3202578, 3202579, 3202580, 3202581, 3...",NaN,"Capahosic, Virginia",NaN,NaN
69,579,Box 1 Folder 25,Draft: Address at the Dedication of W.E.B. Du ...,"Sat, 10/18/1969 - 12:00",17.0,"Draft, Handwritten, Typed",Papers of Julian Bond 1897-2006,"Special Collections, University of Virginia Li...",Scanning Notes: There are two 

It is acceptable to me, for now, to include these locations without latitude and longitude data. But I cannot because of Drupal limitations. Will have to manually enter the other locations.

In [202]:
# year extraction

all_located_speeches['date_mod'] = all_located_speeches.apply(lambda row: row['Document Date'].split('-')[0].strip(' '), axis=1)
all_located_speeches['year'] = all_located_speeches.apply(lambda row: parser.parse(row['date_mod']).year, axis=1)

In [204]:
all_located_speeches['year'].value_counts()

year
1975    21
1977    17
1972    11
1976    10
1973     9
1971     9
1970     9
1981     5
1979     5
1998     5
1980     4
2001     4
1999     4
1989     4
1969     4
1988     3
2003     3
2000     3
1978     3
1991     2
1983     2
1990     2
2004     2
1965     1
1993     1
1987     1
1982     1
1995     1
1996     1
1994     1
2002     1
1992     1
2005     1
1967     1
1974     1
Name: count, dtype: int64

In [207]:
sum(all_located_speeches.year.value_counts())

153

In [209]:
len(all_located_speeches)

153

In [206]:
all_located_speeches.columns

Index(['ID', 'Box', 'title', 'Document Date', 'Document Length',
       'Manuscript Type', 'Collection', 'Source', 'Internal Notes',
       'Workflow Stage', 'Object Type', 'Author', 'Keywords', 'Head',
       'Document Top', 'Dateline', 'Salutation', 'Document Body', 'Signature',
       'Post Script', 'Document Extra', 'Marginalia', 'Enclosure', 'Footnote',
       'General', 'Image Identifier', 'Response', 'city', 'lat', 'lng',
       'date_mod', 'year'],
      dtype='object')

In [210]:
export = all_located_speeches[['ID', 'title', 'Manuscript Type', 'Workflow Stage', 
                               'Keywords', 'city', 'lat', 'lng', 'year']]

In [213]:
export.head()

,ID,title,Manuscript Type,Workflow Stage,Keywords,city,lat,lng,year
0,80,"Speech made before the Hungry Club [Forum, Atl...",NaN,Early Access,NaN,"Atlanta, Georgia",33.7628,-84.4220,1972
1,97,Testimony before the Commission on Delegate Se...,NaN,Early Access,NaN,"Atlanta, Georgia",33.7628,-84.4220,1973
2,96,Testimony before the Commission on Delegate Se...,NaN,Early Access,NaN,"Atlanta, Georgia",33.7628,-84.4220,1973
3,95,Testimony before the Commission on Delegate Se...,NaN,Early Access,NaN,"Atlanta, Georgia",33.7628,-84.4220,1973
4,90,"Speech at the National Urban League Meeting, K...","Typed, Handwritten, Version",Early Access,NaN,"Kansas City, Missouri",39.1238,-94.5541,1973


In [214]:
export.to_csv('tagged_speeches_toArcGIS_20250818.csv')

## Future work

Several components:

1. Exporting a locations table and attempting a locations feed import (highest priority)
2. Completing the Storymap (high priority)
3. Analyzing locations spoken of more generally, looking at more than just titles
4. Extending location search to Series II, which will require analyzing document top, head, dateline, etc.

In [190]:
# 1. Export locations table

locations = all_located_speeches[['ID', 'city', 'lat', 'lng']]

In [191]:
locations = locations.groupby(['city', 'lat', 'lng'])['ID'].agg(list).reset_index()

In [192]:
locations = locations.rename({'ID':'Body', 'city':'Title'}, axis = 1)
locations['Location Map'] = locations.apply(lambda row: str(row['lat']) + ', ' + str(row['lng']), axis = 1)
locations['Body'] = locations['Body'].apply(lambda x: ', '.join(map(str, x)))

In [194]:
locations_export = locations[['Title', 'Body', 'Location Map']]

In [197]:
locations_export['Summary'] = 'IDs of documents identified with this location.'

C:\Users\charl\AppData\Local\Temp\ipykernel_16836\1711520103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  locations_export['Summary'] = 'IDs of documents identified with this location.'


In [200]:
locations_export.to_csv('locations_feed.csv', index = False)